In [1]:
!pip install yfinance
import numpy as np
import pandas as pd
import warnings
import os
import yfinance as yf

     |████████████████████████████████| 6.4 MB 8.1 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
df = yf.download('084680.KS', start = '2015-01-01', end='2019-12-31')

[*********************100%***********************]  1 of 1 completed


In [3]:
df['CODE'] = 'SPY'

In [4]:
price_df = df.loc[:,['Adj Close']].copy()

In [5]:
price_df['center'] = price_df['Adj Close'].rolling(20).mean() # 20일 이동평균선

In [6]:
# +2 표준편차를 둔 상단 밴드
price_df['ub'] = price_df['center'] + 2 * price_df['Adj Close'].rolling(20).std() 

In [7]:
# -2 표준편차를 둔 하단 밴드
price_df['lb'] = price_df['center'] - 2 * price_df['Adj Close'].rolling(20).std()

In [8]:
n = 20 #이평선
sigma = 2 #시그마

In [9]:
def bollinger_band(price_df, n, sigma):
    bb = price_df.copy()
    bb['center'] = price_df['Adj Close'].rolling(n).mean() #중앙 이동평균선
    bb['ub'] = bb['center'] + sigma * price_df['Adj Close'].rolling(n).std() # 상단 밴드
    bb['lb'] = bb['center'] - sigma * price_df['Adj Close'].rolling(n).std() # 하단 밴드
    return bb

In [10]:
bollinger = bollinger_band(price_df, n, sigma)

In [11]:
base_date = '2009-01-02'

In [12]:
sample = bollinger.loc[base_date:]

In [13]:
book = sample[['Adj Close']].copy()

In [14]:
book['trade'] = '' #거래내역 컬럼

In [15]:
def create_trade_book(sample):
    book = sample[['Adj Close']].copy()
    book['trade'] = ''
    return (book)    

In [16]:
book = create_trade_book(sample)

In [17]:
def tradings(sample, book):
    for i in sample.index:
        if sample.loc[i, 'Adj Close'] > sample.loc[i, 'ub']: # 상단밴드 이탈시 동작 안함
            book.loc[i, 'trade'] = ''
        elif sample.loc[i, 'lb'] > sample.loc[i, 'Adj Close']: # 하반밴드 이탈시 매수
            if book.shift(1).loc[i, 'trade'] == 'buy':    # 이미 매수상태라면
                book.loc[i, 'trade'] = 'buy'     # 매수상태 유지
            else:
                book.loc[i, 'trade'] = 'buy'    
        elif sample.loc[i, 'ub'] >= sample.loc[i, 'Adj Close'] and sample.loc[i, 'Adj Close'] >= sample.loc[i, 'lb']: # 볼린저 밴드 안에 있을 시
            if book.shift(1).loc[i, 'trade'] == 'buy':
                book.loc[i, 'trade'] = 'buy'  # 매수상태 유지
            else:
                book.loc[i, 'trade'] = '' # 동작 안함
    return (book) 

In [18]:
book = tradings(sample,book)

In [19]:
for i in book.index:
    if book.loc[i, 'trade'] == 'buy' and book.shift(1).loc[i, 'trade'] == '':     # long 진입
        buy = book.loc[i, 'Adj Close']
        print('진입일 : ',i, 'long 진입가격 : ', buy)
    elif book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy':     # long 청산
        sell = book.loc[i, 'Adj Close']
        rtn = (sell - buy) / buy + 1 # 손익 계산
        book.loc[i, 'return'] = rtn
        print('청산일 : ',i, 'long 진입가격 : ', buy, ' |  long 청산가격 : ', \
              sell, ' | return:', round(rtn, 4))

    if book.loc[i, 'trade'] == '':     # zero position
        buy = 0.0
        sell = 0.0

진입일 :  2015-07-09 00:00:00 long 진입가격 :  1970.0
청산일 :  2015-09-21 00:00:00 long 진입가격 :  1970.0  |  long 청산가격 :  2310.0  | return: 1.1726
진입일 :  2015-12-08 00:00:00 long 진입가격 :  2670.0
청산일 :  2016-01-19 00:00:00 long 진입가격 :  2670.0  |  long 청산가격 :  2825.0  | return: 1.0581
진입일 :  2016-03-31 00:00:00 long 진입가격 :  2760.0
청산일 :  2016-06-07 00:00:00 long 진입가격 :  2760.0  |  long 청산가격 :  3015.0  | return: 1.0924
진입일 :  2016-09-05 00:00:00 long 진입가격 :  2660.0
청산일 :  2016-12-15 00:00:00 long 진입가격 :  2660.0  |  long 청산가격 :  2020.0  | return: 0.7594
진입일 :  2017-03-03 00:00:00 long 진입가격 :  2020.0
청산일 :  2017-03-24 00:00:00 long 진입가격 :  2020.0  |  long 청산가격 :  2470.0  | return: 1.2228
진입일 :  2017-06-21 00:00:00 long 진입가격 :  1925.0
청산일 :  2017-09-13 00:00:00 long 진입가격 :  1925.0  |  long 청산가격 :  1975.0  | return: 1.026
진입일 :  2017-12-08 00:00:00 long 진입가격 :  1790.0
청산일 :  2018-01-26 00:00:00 long 진입가격 :  1790.0  |  long 청산가격 :  1845.0  | return: 1.0307
진입일 :  2018-05-25 00:00:00 long 진입가격 :  1805.0
청산

In [20]:
def returns(book):
    # 손익 계산
    rtn = 1.0
    book['return'] = 1
    buy = 0.0
    sell = 0.0
    for i in book.index:
        if book.loc[i, 'trade'] == 'buy' and book.shift(1).loc[i, 'trade'] == '':     # long 진입
            buy = book.loc[i, 'Adj Close']
            print('진입일 : ',i, 'long 진입가격 : ', buy)
        elif book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy':     # long 청산
            sell = book.loc[i, 'Adj Close']
            rtn = (sell - buy) / buy + 1 # 손익 계산
            book.loc[i, 'return'] = rtn
            print('청산일 : ',i, 'long 진입가격 : ', buy, ' |  long 청산가격 : ', \
                  sell, ' | return:', round(rtn, 4))
    
        if book.loc[i, 'trade'] == '':     # zero position
            buy = 0.0
            sell = 0.0
    
    acc_rtn = 1.0
    for i in book.index:
        rtn = book.loc[i, 'return']
        acc_rtn = acc_rtn * rtn  # 누적 수익률 계산
        book.loc[i, 'acc return'] = acc_rtn

    print ('Accunulated return :', round(acc_rtn, 4))
    return (round(acc_rtn, 4))

In [21]:
print(returns(book))

진입일 :  2015-07-09 00:00:00 long 진입가격 :  1970.0
청산일 :  2015-09-21 00:00:00 long 진입가격 :  1970.0  |  long 청산가격 :  2310.0  | return: 1.1726
진입일 :  2015-12-08 00:00:00 long 진입가격 :  2670.0
청산일 :  2016-01-19 00:00:00 long 진입가격 :  2670.0  |  long 청산가격 :  2825.0  | return: 1.0581
진입일 :  2016-03-31 00:00:00 long 진입가격 :  2760.0
청산일 :  2016-06-07 00:00:00 long 진입가격 :  2760.0  |  long 청산가격 :  3015.0  | return: 1.0924
진입일 :  2016-09-05 00:00:00 long 진입가격 :  2660.0
청산일 :  2016-12-15 00:00:00 long 진입가격 :  2660.0  |  long 청산가격 :  2020.0  | return: 0.7594
진입일 :  2017-03-03 00:00:00 long 진입가격 :  2020.0
청산일 :  2017-03-24 00:00:00 long 진입가격 :  2020.0  |  long 청산가격 :  2470.0  | return: 1.2228
진입일 :  2017-06-21 00:00:00 long 진입가격 :  1925.0
청산일 :  2017-09-13 00:00:00 long 진입가격 :  1925.0  |  long 청산가격 :  1975.0  | return: 1.026
진입일 :  2017-12-08 00:00:00 long 진입가격 :  1790.0
청산일 :  2018-01-26 00:00:00 long 진입가격 :  1790.0  |  long 청산가격 :  1845.0  | return: 1.0307
진입일 :  2018-05-25 00:00:00 long 진입가격 :  1805.0
청산

In [25]:
pip install backtrader

     |████████████████████████████████| 410 kB 5.0 MB/s 


In [31]:
import backtrader as bt

In [35]:
import datetime

In [32]:
class BBand_Strategy(bt.Strategy):
    params = (('period', 20),
              ('devfactor', 2.0),)

    def __init__(self):
        # keep track of close price in the series
        self.data_close = self.datas[0].close
        self.data_open = self.datas[0].open

        # keep track of pending orders/buy price/buy commission
        self.order = None
        self.price = None
        self.comm = None

        # add Bollinger Bands indicator and track the buy/sell signals
        self.b_band = bt.ind.BollingerBands(self.datas[0], period=self.p.period, devfactor=self.p.devfactor)
        self.buy_signal = bt.ind.CrossOver(self.datas[0], self.b_band.lines.bot)
        self.sell_signal = bt.ind.CrossOver(self.datas[0], self.b_band.lines.top)

    def log(self, txt):
        '''Logging function'''
        dt = self.datas[0].datetime.date(0).isoformat()
        print(f'{dt}, {txt}')

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # order already submitted/accepted - no action required
            return

        # report executed order
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    f'BUY EXECUTED --- Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f}, Commission: {order.executed.comm:.2f}'
                )
                self.price = order.executed.price
                self.comm = order.executed.comm
            else:
                self.log(
                    f'SELL EXECUTED --- Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f}, Commission: {order.executed.comm:.2f}'
                )

        # report failed order
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Failed')

        # set no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log(f'OPERATION RESULT --- Gross: {trade.pnl:.2f}, Net: {trade.pnlcomm:.2f}')

    def next_open(self):
        if not self.position:
            if self.buy_signal > 0:
                # calculate the max number of shares ('all-in')
                size = int(self.broker.getcash() / self.datas[0].open)
                # buy order
                self.log(f'BUY CREATED --- Size: {size}, Cash: {self.broker.getcash():.2f}, Open: {self.data_open[0]}, Close: {self.data_close[0]}')
                self.buy(size=size)
        else:
            if self.sell_signal < 0:
                # sell order
                self.log(f'SELL CREATED --- Size: {self.position.size}')
                self.sell(size=self.position.size)

In [36]:
data = bt.feeds.YahooFinanceData(dataname='MSFT',
                                 fromdate=datetime.datetime(2018, 1, 1),
                                 todate=datetime.datetime(2018, 12, 31))

In [37]:
# Create a cerebro entity
cerebro = bt.Cerebro(stdstats = False, cheat_on_open=True)

# set up the backtest
cerebro.addstrategy(BBand_Strategy)
cerebro.adddata(data)
cerebro.broker.setcash(10000.0)
cerebro.broker.setcommission(commission=0.001)
cerebro.addobserver(MyBuySell)
cerebro.addobserver(bt.observers.Value)
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='time_return')


print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
backtest_result = cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.plot(iplot=True, volume=False)

# returns analyzer
print(backtest_result[0].analyzers.returns.get_analysis())

# returns over time analyzer
#%%
# plt.style.use('seaborn') #set style to `seaborn`
returns_dict = backtest_result[0].analyzers.time_return.get_analysis()
pd.DataFrame(list(returns_dict.items()), columns = ['report_date', 'return']).set_index('report_date').plot()


NameError: ignored

In [ ]:
import matplotlib.pylab as plt

In [ ]:
book['acc return'].plot()